# Package import

In [15]:
import csv
import numpy as np
import pandas as pd
import pyfpgrowth
import time
from datetime import timedelta
from PrefixSpan import *

# 1. Read Data

In [16]:
# only use the new data
raw_data = pd.read_csv('../../data/trade_new.csv', index_col=0) # trade_new.csv  and  trade.csv
raw_data = raw_data.reset_index(drop=True)


In [ ]:
plu_list = raw_data[['pluno', 'pluname']]
plu_dict = {}
for i in range(len(plu_list)):
    pluno = plu_list.loc[i,'pluno']
    pluname = plu_list.loc[i,'pluname']
    plu_dict[pluno] = pluname
plu_list

In [17]:
# use a dict to store the transaction of each vip
# also record the sldatime
l_data = len(raw_data)
vip_transaction = {}
vip_set = set()
for i in range(l_data):
    v = raw_data.loc[i, 'vipno']
    t = raw_data.loc[i, 'sldatime'] # sldat -> trade.csv   sldatime -> trade_new.csv
    if v in vip_set:
        vip_transaction[v].append((t, i))
    else:
        vip_transaction[v] = [(t, i)]
        vip_set.add(v)
        
        
vip_list = list(vip_set)
l_list = []
with open("item_no.csv", 'w') as f:
    for v in vip_list:
        l = len(vip_transaction[v])
        l_list.append(l)
        f.write('{}\n'.format(l))
print(np.mean(l_list))

46.5


In [18]:
target_vip = []
for v in vip_list:
    if len(vip_transaction[v])>33:
        target_vip.append(v)

In [19]:
# sort and cut the data
vip_train = {}
vip_test = {}
for v in vip_list:
    # sort
    t = vip_transaction[v]
    t = sorted(t, key=lambda x:x[0])
    
    # cut and pick up the index only
    l = round(len(t) * 0.6)
    train = t[:l]
    test = t[l:]
    vip_train[v] = [tt[1] for tt in train] # tt[0] --> time , tt[1] --> index of raw_data
    vip_test[v] = [tt[1] for tt in test]

In [20]:
def split_transaction(indice, item_label, transactions):
    split_data = raw_data.loc[indice, ['uid', item_label]] # get the uid and pluno/dptno/bndno columns
    if item_label == 'bndno':
        split_data = split_data.dropna(how='any')
        split_data['bndno'] = split_data['bndno'].astype('int')
    groups = split_data.groupby('uid').groups
    for u in groups:
        index = groups[u] # index of the same uid
        t = list(split_data.loc[index, item_label]) # get one transaction
        t = [[x] for x in t]
        transactions.append(t)

In [29]:
# mining the sequence pattern first
item_label = 'pluno'
print("item -- {}".format(item_label))
transactions = []
for v in target_vip:
    split_transaction(vip_train[v], item_label, transactions)
print(len(transactions))
supports = [10, 8, 6, 4]
patterns = {}
for s in supports:
    time_start=time.time()
    p = prefixSpan(SquencePattern([], sys.maxsize), transactions, s)
#     patterns = pyfpgrowth.find_frequent_patterns(transactions, s)
    pp = []
    for x in p:
        if len(x.squence) > 1:
            pp.append(x)
    patterns[s] = pp
    time_end=time.time()
    print('support = {} done. totally cost = '.format(s), timedelta(seconds=time_end-time_start))

item -- pluno
2171
support = 10 done. totally cost =  0:00:00.520038
support = 8 done. totally cost =  0:00:00.963250
support = 6 done. totally cost =  0:00:00.929649
support = 4 done. totally cost =  0:00:02.194567


In [77]:
for s in patterns:
    print(len(patterns[s]))
t = set()
for x in patterns[4]:
    for i in x.squence:
        t.add(len(i))
print(t)

27
52
116
340
{1}


In [123]:
pattern = patterns[10]

In [124]:
train_set = {}
for v in target_vip:
    purchase = []
    indice = vip_train[v] # the purchase record indice
    split_data = raw_data.loc[indice, ['uid', 'pluno']] # get the uid and pluno/dptno/bndno columns
    groups = split_data.groupby('uid').groups
    for u in groups:
        index = groups[u] # index of the same uid
        t = list(split_data.loc[index, 'pluno']) # get one transaction
        purchase.append(t)
    train_set[v] = purchase
    
test_set = {}
for v in target_vip:
    purchase = []
    indice = vip_test[v] # the purchase record indice
    split_data = raw_data.loc[indice, 'pluno']
    test_set[v] = list(split_data)

In [125]:
# print(train_set[target_vip[1]])
# print(test_set[target_vip[1]])
f_pattern = []
remove_list = [30380002, 30380001, 30380003]
for p in pattern:
    t = [x[0] for x in p.squence]
    flag = True
    for plastic in remove_list:
        if plastic in t:
            flag= False
    if flag:
        f_pattern.append(t)
        pattern_plu = [plu_dict[x] for x in t]
        print(pattern_plu)
# print(f_pattern)

['炒菜系列', '炒菜系列']
['去叶大葱', '西红柿']
['黄瓜', '西红柿']
['散馒头', '散馒头']
['奶油球', '奶油球']
['八分场凉皮', '八分场凉皮']
['大面包', '大面包']


In [ ]:
reco = {}
for v in target_vip:
    print("vip = ", v)
    reco[v] = []
    shopping_list = train_set[v]
    for p in f_pattern:
        p_l = len(p)
        p_i = 0
        s_l = len(shopping_list)
        s_i = 0
        while p_i < p_l and s_i < s_l:
            if (p[p_i] in shopping_list[s_i]):
                p_i += 1
            s_i += 1
        if p_i == p_l - 1:
            reco[v].append(p[p_i])
        tem = [plu_dict[x] for x in reco[v]]
    print("recommend -- len={0}, list: {1}".format(len(reco[v]), tem))

In [ ]:
precision = []
recall = []
for v in target_vip:
    print("vip = ", v)
    hit = 0
    reco_l = len(reco[v])
    shop_l = len(test_set[v])
    if reco_l == 0:
        continue
    for item in reco[v]:
        if item in test_set[v]:
            hit += 1
    precision.append(hit/reco_l)
    recall.append(hit/shop_l)
    print("presicion = {0} / {1}".format(hit, reco_l))
    print("recall = {0} / {1}".format(hit, shop_l))
print(np.mean(precision))
print(np.mean(recall))